In [2]:
import librosa
import librosa.display
import os
import pickle
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import re
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.under_sampling import ClusterCentroids
from xgboost import XGBClassifier

import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

In [3]:
with open('X_train_4.pkl', 'rb') as f:
    X_train = pickle.load(f)

In [4]:
with open('y_train_4.pkl', 'rb') as f:
    y_train = pickle.load(f)

In [5]:
with open('X_val_4.pkl', 'rb') as f:
    X_val = pickle.load(f)

In [6]:
with open('y_val_4.pkl', 'rb') as f:
    y_val = pickle.load(f)

In [7]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_val = (X_val - mean)/std

## GB Classifier

In [8]:
y_train = y_train.argmax(axis=1)
y_val = y_val.argmax(axis=1)

In [9]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [10]:
print(clf.score(X_train, y_train))
print(clf.score(X_val, y_val))

0.9550898203592815
0.27102803738317754


In [11]:
y_pred = clf.predict(X_val)

In [12]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.01      0.04      0.02        26
           1       0.02      0.04      0.02        25
           2       0.29      0.34      0.31       179
           3       0.60      0.27      0.37       441
           4       0.11      0.31      0.16        78

    accuracy                           0.27       749
   macro avg       0.21      0.20      0.18       749
weighted avg       0.43      0.27      0.31       749



In [13]:
print(confusion_matrix(y_val, y_pred))

[[  1   3   6   6  10]
 [  1   1   8  10   5]
 [ 17  13  60  49  40]
 [ 43  31 112 117 138]
 [  7  12  21  14  24]]


In [14]:
lb = LabelEncoder()
with open('labels', 'rb') as f:
    lb = pickle.load(f)

In [15]:
lb.inverse_transform([0,1,2,3,4])

array(['disgust', 'fear', 'happy', 'neutral', 'sad'], dtype=object)

In [16]:
clf2 = XGBClassifier(max_depth =25,n_estimators = 20000,eta = 0.07,objective = "multi:softmax",num_class = 5)
clf2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.07, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=25,
              min_child_weight=1, missing=None, n_estimators=20000, n_jobs=1,
              nthread=None, num_class=5, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [17]:
print(clf2.score(X_train, y_train))
print(clf2.score(X_val, y_val))

1.0
0.2870493991989319


In [18]:
y_pred2 = clf2.predict(X_val)

In [19]:
print(classification_report(y_val, y_pred2))

              precision    recall  f1-score   support

           0       0.05      0.15      0.08        26
           1       0.00      0.00      0.00        25
           2       0.31      0.35      0.33       179
           3       0.65      0.27      0.38       441
           4       0.13      0.36      0.19        78

    accuracy                           0.29       749
   macro avg       0.23      0.23      0.20       749
weighted avg       0.47      0.29      0.33       749

